# Imports

In [ ]:
# Suprress warnings
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    # Standard imports
    import matplotlib
    import matplotlib.pyplot as plt
    import matplotlib.patches as patches
    import matplotlib.cm as cm
    import numpy as np

    # Drake imports
    from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
    import pydrake
    from pydrake.all import LogOutput, DirectCollocation, DirectTranscription, MathematicalProgram, InputPortSelection
    from pydrake.all import FindResourceOrThrow
    
    import scipy.interpolate
# Imports of other project files
from log_wrapper import LogWrapper
import arm
import constants
import finger
import pedestal
from controllers import edge_controller
from paper import Paper

from pydrake.all import (MultibodyPlant, Parser, DiagramBuilder, Simulator, RigidTransform,
                         PlanarSceneGraphVisualizer, SceneGraph, TrajectorySource,
                         SnoptSolver, MultibodyPositionToGeometryPose, PiecewisePolynomial,
                         MathematicalProgram, JacobianWrtVariable, eq, RollPitchYaw, AutoDiffXd, BodyIndex,
                        RotationMatrix)

# Other imports
import importlib

In [ ]:
from IPython.display import display, SVG
import pydot

In [ ]:
# Matplotlib configuring
# USE FOR PAPER
# plt.style.use(['science', 'no-latex'])
# font = {'size'   : 14}
# matplotlib.rc('font', **font)
# USE GENERALLY
font = {'size'   : 16}
matplotlib.rc('font', **font)

In [ ]:
# Meshcat init
proc, zmq_url, web_url = start_zmq_server_as_subprocess()

# Simulation setup
## Pre-finalize steps

In [ ]:
builder = pydrake.systems.framework.DiagramBuilder()

jnt_frc_log = []

# Add all elements
plant, scene_graph = pydrake.multibody.plant.AddMultibodyPlantSceneGraph(builder, time_step=constants.DT)
v_stiction=1e-3
plant.set_stiction_tolerance(v_stiction)
plant.set_penetration_allowance(0.001)
pedestal_instance = pedestal.AddPedestal(plant)

# These joint angles start the paper approximately the right spot, no matter how many links are used
num_links = 2
def_joint_angles = 0

paper = Paper(plant, scene_graph, num_links, default_joint_angle=def_joint_angles,
              stiffness=2.5e-1,
              damping=7.12547340446979e-06)
paper.weld_paper_edge(pedestal.PEDESTAL_WIDTH, pedestal.PEDESTAL_HEIGHT)

# Set up logger (needs to happen after all bodies are added)
# log_wrapper = LogWrapper(plant.num_bodies(), 0, paper, jnt_frc_log) # int(finger_body.index())
# builder.AddSystem(log_wrapper)

Calculate link inertia to use in damping calculations:

In [ ]:
arm_instance = arm.AddArm(plant, scene_graph)

## Controller selection

In [ ]:
# I_L = plant.get_body(
#     BodyIndex(paper.link_idxs[-1])).default_rotational_inertia().CalcPrincipalMomentsOfInertia()[0]
# I_M = finger_body.default_rotational_inertia().CalcPrincipalMomentsOfInertia()[0]
# sys_params = {
#     'I_L': I_L,
#     'I_M': I_M,
#     'v_stiction': v_stiction,
#     'w_L': paper.link_width,
#     'm_L': paper.link_mass,
#     'b_J': paper.damping,
#     'k_J': paper.stiffness,
#     'g': plant.gravity_field().gravity_vector()[-1]*-1,
# }
# finger_ctrlr = edge_controller.EdgeController(
#     finger_idx=int(finger_body.index()),
#     ll_idx=int(paper.link_idxs[-1]),
#     sys_params=sys_params,
#     debug=True,
#     jnt_frc_log=jnt_frc_log
# )

In [ ]:
arm_ctrl = arm.ArmForceController()

## Post-finalize steps

In [ ]:
plant.Finalize()

In [ ]:
## Post finalize steps
# Conect finger controller
# builder.AddSystem(finger_ctrlr)
# builder.Connect(finger_ctrlr.get_output_port(), plant.get_actuation_input_port(finger_instance))
# builder.Connect(plant.get_body_poses_output_port(), finger_ctrlr.get_input_port(0))
# builder.Connect(plant.get_body_spatial_velocities_output_port(), finger_ctrlr.get_input_port(1))
# builder.Connect(plant.get_contact_results_output_port(), finger_ctrlr.get_input_port(2))

# Add logger
# builder.Connect(plant.get_body_poses_output_port(), log_wrapper.get_input_port(0))
# builder.Connect(plant.get_body_spatial_velocities_output_port(), log_wrapper.get_input_port(1))
# builder.Connect(plant.get_body_spatial_accelerations_output_port(), log_wrapper.get_input_port(2)) 
# builder.Connect(plant.get_contact_results_output_port(), log_wrapper.get_input_port(3))
# builder.Connect(plant.get_reaction_forces_output_port(), log_wrapper.get_input_port(4))
builder.AddSystem(arm_ctrl)
builder.Connect(plant.get_state_output_port(arm_instance), arm_ctrl.get_input_port())
builder.Connect(arm_ctrl.get_output_port(), plant.get_actuation_input_port())



# Visualization and logging
# logger = LogOutput(log_wrapper.get_output_port(), builder)
vis = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder, scene_graph)
# vis.set_planar_viewpoint(camera_position=[1, 0, 0], xmin=-0.3, xmax=0.3, ymin=-0.3, ymax=0.3)

# Build diagram and do actions requiring 
diagram = builder.Build()
diagram_context = diagram.CreateDefaultContext()

In [ ]:
q0 = np.zeros(7)
q0[0] = -np.pi/2
q0[1] = 1.1
q0[3] = -2
q0[5] = 3*np.pi/2-0.2
q0[6] = -np.pi/4
plant_context = diagram.GetMutableSubsystemContext(plant, diagram_context)
plant.SetPositions(plant_context, arm_instance, q0)

In [ ]:
# SVG(pydot.graph_from_dot_data(diagram.GetGraphvizString())[0].create_svg())

# Run simulation

In [ ]:
# Finalize simulation and visualization
simulator = pydrake.systems.analysis.Simulator(diagram, diagram_context)
simulator.Initialize()
vis.start_recording()
try:
    simulator.AdvanceTo(constants.TSPAN)
except RuntimeError as e:
    print(e)

# # This way, we can use the actual time the simulation ran for
# # effective_tspan = max(logger.sample_times())

vis.stop_recording()
vis.publish_recording()

In [ ]:
ajflkdsjfakljsdklf

# Plots used in orginal paper

In [ ]:
# This script is getting huge. We need to turn off parts that we're not using.
generate_orig_plots = True

In [ ]:
if generate_orig_plots:
    # Generate plots for paper
    nb = plant.num_bodies()
    y_traces = []
    z_traces = []
    theta_traces = []
    for b in paper.link_idxs:
        y_traces.append(logger.data()[log_wrapper.entries_per_body*b+1])
        z_traces.append(logger.data()[log_wrapper.entries_per_body*b+2])
        theta_traces.append(logger.data()[log_wrapper.entries_per_body*b+3])
    y_traces = np.array(y_traces)
    z_traces = np.array(z_traces)
    theta_traces = np.array(theta_traces)

    times_ = np.arange(0,effective_tspan, effective_tspan/10)
    cmap = cm.get_cmap("viridis_r")
    plt.figure(figsize=(2*3,2*2))
    for t in times_:
        c = cmap(t/effective_tspan)
        idx = np.argmax(logger.sample_times() >= t)

        # Plot paper
        # PROGRAMMING: Account for paper thickness in plots
        for y, z, theta in zip(y_traces[:,idx], z_traces[:,idx], theta_traces[:,idx]):
            y0 = y - np.cos(theta)*paper.link_width/2
            z0 = z - np.sin(theta)*paper.link_width/2
            y1 = y + np.cos(theta)*paper.link_width/2
            z1 = z + np.sin(theta)*paper.link_width/2
            plt.plot([y0, y1], [z0, z1], color=c)


        # Plot manipulator
        plt.scatter(logger.data()[log_wrapper.entries_per_body*int(finger_instance)+1,idx], 
                    logger.data()[log_wrapper.entries_per_body*int(finger_instance)+2,idx],
                    color=c, s=300, zorder=1)

    xlim = plt.xlim()
    ylim = plt.ylim()
    plt.scatter([xlim[0]-50, xlim[0]-50], [ylim[0]-50, ylim[0]-50], c=[0, effective_tspan], cmap=cmap)
    plt.xlim(xlim)
    plt.ylim(ylim)
    cb = plt.colorbar()
    cb.set_label("Time")
    plt.xlabel("$y$ position")
    plt.ylabel("$z$ position")
    plt.show()

# Drag plots

This is a sanity check to see how much force we're neglecting by ignoring drag. The colored lines are the drag forces at each link, and the dashed black line is the force due to gravity for scale.

Note that the formula for drag force is $\rho v^2 C_d A_{paper}$.

$C_d$ comes from [this site](https://www.engineersedge.com/fluid_flow/rectangular_flat_plate_drag_14036.htm), which says it depends on $L/d$. For 20 links, $L/d\approx15$. Rounding this down to 10, we get $C_d=1.22$.

In [ ]:
generate_drag_plots = False

In [ ]:
if generate_drag_plots:
    # Plot drag forces
    plt.figure(figsize=(16, 12))
    nb = plant.num_bodies
    vel_traces = []
    for b in paper.link_idxs:
        rho = 1.225
        C_d = paper.link_width*paper.depth
        vel_sqred = logger.data()[log_wrapper.entries_per_body*b+7]**2+logger.data()[log_wrapper.entries_per_body*b+8]**2
        f = 0.5*rho*C_d*vel_sqred
        plt.plot(f)

    plt.axhline(paper.link_mass*9.81, color='black', linestyle='--')
    plt.show()

# Verifying equations hold
We have the following free body diagrams:
<img src="forces.png" width=400px>
<img src="link-moments.png" width=400px>

Which give us the following equations:
$$
\begin{aligned}
     m_La_{LT} &= F_{FL} + F_{GT} +F_{OT} \\
    m_La_{LN} &= F_{NL} + F_{GN} +F_{ON} \\
    m_Ma_{MT} &= F_{FM} + F_{CT} \\
    m_Ma_{MN} &= F_{NM} + F_{CN} \\
    a_{LT} &= a_{MT} \\
    a_{LN} &= a_{MN} \\
    a_{LT} &= -\frac{w_L}{2}\dot\theta^2 \\
    a_{LN} &= \frac{w_L}{2}\ddot\theta \\
    I_L\ddot\theta &= \frac{w_L}{2}F_{ON}- \frac{h_L}{2}F_{FL} - r_TF_{NL} \\
    F_{NL} &= -F_{NM} \\
    F_{FL} &= -F_{FM} \\
    a_{NL} &= a_{Nd}  \\
    F_{FL} &= 0
\end{aligned}
$$
For each of these equations, I want to plot the value on the left side and the value on the right side to see if they match.

## Loading signals
Preparing all the signals we'll need.

In [ ]:
debug = finger_ctrlr.debug

In [ ]:
l_times = logger.sample_times()
d_times = np.array(debug['times'])

### Helper functions

In [ ]:
def get_N_proj(vec):
    N_vec = np.matmul(N_hat_proj_mat, vec)
    N_mag = np.linalg.norm(N_vec, axis=1)
    N_sgn = np.sign(np.matmul(np.transpose(N_hats, [0, 2, 1]), N_vec))
    N = N_mag.flatten()*N_sgn.flatten()
    return N

In [ ]:
def get_T_proj(vec):
    T_vec = np.matmul(T_hat_proj_mat, vec)
    T_mag = np.linalg.norm(T_vec, axis=1)
    T_sgn = np.sign(np.matmul(np.transpose(T_hats, [0, 2, 1]), T_vec))
    T = T_mag.flatten()*T_sgn.flatten()
    return T

In [ ]:
def diff_vec(vec):
    d_vec = np.diff(vec, axis=0)
    for i in range(vec.shape[1]):
        d_vec[:, i] /= np.expand_dims(np.diff(l_times), 1)
    d_vec = np.concatenate((d_vec, [d_vec[-1]]))
    return d_vec

In [ ]:
def plot_vec(sig1, sig1_label, sig2, sig2_label, finish_plotting=True, plot_x=True):
    plt.figure(figsize=(16, 8))
    lw = 5
    if plot_x:
        plt.plot(l_times, sig1[:,0], label=sig1_label + r' $\hat x$ component', color='pink', linewidth=lw)
    plt.plot(l_times, sig1[:,1], label=sig1_label + r' $\hat y$ component', color='lightgreen', linewidth=lw)
    plt.plot(l_times, sig1[:,2], label=sig1_label + r' $\hat z$ component', color='lightskyblue', linewidth=lw)
    
    if plot_x:
        plt.plot(l_times, sig2[:,0], label=sig2_label + r' $\hat x$ component', color='red', linewidth=lw, linestyle='--')
    plt.plot(l_times, sig2[:,1], label=sig2_label + r' $\hat y$ component', color='green', linewidth=lw, linestyle='--')
    plt.plot(l_times, sig2[:,2], label=sig2_label + r' $\hat z$ component', color='blue', linewidth=lw, linestyle='--')
    plt.xlabel("Time (seconds)")
    plt.legend()
    plt.autoscale(enable=True, axis='x', tight=True)
    if finish_plotting:
        plt.show()

In [ ]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'same') / w

In [ ]:
def moving_average_vec(x, w):
    out = np.zeros_like(x)
    for i in range(x.shape[1]):
        out[:,i] = np.expand_dims(moving_average(x[:,i].flatten(), w), 1)
    return out

In [ ]:
def diff_arr(arr):
    d_arr = np.diff(arr)/np.diff(l_times)
    d_arr = np.concatenate((d_arr, [d_arr[-1]]))
    return d_arr

In [ ]:
def step5(x):
    '''Python version of MultibodyPlant::StribeckModel::step5 method'''
    x3 = x * x * x
    return x3 * (10 + x * (6 * x - 15))

def stribeck(us, uk, v):
    '''
    Python version of MultibodyPlant::StribeckModel::ComputeFrictionCoefficient
    
    From
    https://github.com/RobotLocomotion/drake/blob/b09e40db4b1c01232b22f7705fb98aa99ef91f87/multibody/plant/images/stiction.py
    '''
    u = np.zeros_like(v) + uk
    u[v < 1] = us * step5(v[v < 1])
    mask = (v >= 1) & (v < 3)
    u[mask] = us - (us - uk) * step5((v[mask] - 1) / 2)
    return u

### Inputs
These should all come from simulation/geometry, since they are inputs to the controller and are "fixed" from the controller's perspective.
#### Geometric quantities ($m_L, m_M, w_L, I_L, h_L, \mu$)

In [ ]:
m_L = paper.link_mass
m_M = finger.MASS
w_L = paper.link_width
I_L = paper.plant.get_body(
    BodyIndex(paper.link_idxs[-1])).default_rotational_inertia().CalcPrincipalMomentsOfInertia()[0]
I_M = finger_body.default_rotational_inertia().CalcPrincipalMomentsOfInertia()[0]
h_L = paper.height
mu = constants.FRICTION
r = finger.RADIUS
b_J = paper.damping
k_J = paper.stiffness

#### Vectors which aren't determined by the force balance ($F_{GT}, F_{GN} \hat T, \hat N$)
- [ ] PROGRAMMING: also get $\hat T$ from simulation somehow

In [ ]:
N_hats_raw = logger.data()[log_wrapper.contact_entry_start_idx+14:log_wrapper.contact_entry_start_idx+17,:]
N_hats = np.expand_dims(N_hats_raw.T, 2)

In [ ]:
T_hats = np.matmul(
    np.array([
        [0,  0, 0],
        [0,  0, 1],
        [0, -1, 0],
    ]),
    N_hats)

It's also useful here to calculate project matrices:

In [ ]:
T_hat_proj_mat = np.matmul(T_hats, np.transpose(T_hats, [0, 2, 1]))

In [ ]:
N_hat_proj_mat = np.matmul(N_hats, np.transpose(N_hats, [0, 2, 1]))

In [ ]:
g = plant.gravity_field().gravity_vector()[-1]*-1
F_G = np.zeros([l_times.size, 3, 1])
F_G[:,2,0] = -np.ones_like(l_times)*g*m_L
F_GT = get_T_proj(F_G)
F_GN = get_N_proj(F_G)

#### Positions and velocities which are "fixed'' until the next timestep ($\dot\theta^2, r_T, \dot d_T$)

##### Poses and velocities of objects

In [ ]:
contact_point = logger.data()[log_wrapper.contact_entry_start_idx+11:log_wrapper.contact_entry_start_idx+14,:]

In [ ]:
p_C = np.expand_dims(contact_point.T, 2)
p_CT = get_T_proj(p_C)
p_CN = get_N_proj(p_C)

In [ ]:
p_L = np.expand_dims(
    logger.data()[log_wrapper.entries_per_body*paper.link_idxs[-1]:
                  log_wrapper.entries_per_body*paper.link_idxs[-1]+3].T,
    2)
p_LN = get_N_proj(p_L)
p_LT = get_T_proj(p_L)

In [ ]:
theta_L = logger.data()[log_wrapper.entries_per_body*paper.link_idxs[-1] + 3]

In [ ]:
v_L = np.expand_dims(
    logger.data()[log_wrapper.entries_per_body*paper.link_idxs[-1]+6:
                  log_wrapper.entries_per_body*paper.link_idxs[-1]+6+3].T,
    2)
v_LN = get_N_proj(v_L)
v_LT = get_T_proj(v_L)

In [ ]:
p_M = np.expand_dims(logger.data()[log_wrapper.entries_per_body*int(finger_body.index()):
                    log_wrapper.entries_per_body*int(finger_body.index())+3,:].T, 2)
p_MN = get_N_proj(p_M)
p_MT = get_T_proj(p_M)

In [ ]:
v_M = np.expand_dims(
    logger.data()[log_wrapper.entries_per_body*int(finger_body.index())+6:
                  log_wrapper.entries_per_body*int(finger_body.index())+6+3].T,
    2)
v_MN = get_N_proj(v_M)
v_MT = get_T_proj(v_M)

In [ ]:
slip_speed = logger.data()[log_wrapper.contact_entry_start_idx+10,:]

##### Directly measured points and velocities

In [ ]:
# Link edge
p_LE = p_L + (w_L/2)*T_hats-(h_L/2)*N_hats
p_LEN = get_N_proj(p_LE)
p_LET = get_T_proj(p_LE)

In [ ]:
# Penetration depth
pen_depth = logger.data()[log_wrapper.contact_entry_start_idx+17]
pen_vec = np.expand_dims(pen_depth, [1,2])*N_hats

In [ ]:
# d vector
d_vec = p_C - p_LE + pen_vec/2
d_T = get_T_proj(d_vec)
d_N = get_N_proj(d_vec)

In [ ]:
d_theta_L = logger.data()[log_wrapper.entries_per_body*paper.link_idxs[-1] + 6 + 3]
d_theta_M = logger.data()[log_wrapper.entries_per_body*int(finger_body.index()) + 6 + 3]

In [ ]:
d_d_T = -d_theta_L*h_L/2-d_theta_L*r - v_LT + v_MT + d_theta_L*d_N
d_d_N = -d_theta_L*w_L/2-v_LN+v_MN-d_theta_L*d_T

In [ ]:
dd_d_T = diff_arr(d_d_T)
dd_d_N = diff_arr(d_d_N)

In [ ]:
d_d_vec = diff_vec(d_vec)
d_d_vec_T = get_T_proj(d_d_vec)
d_d_vec_N = get_N_proj(d_d_vec)

In [ ]:
omega_vec_L = np.expand_dims(np.vstack((d_theta_L, np.zeros_like(d_theta_L), np.zeros_like(d_theta_L))).T, 2)
omega_vec_M = np.expand_dims(np.vstack((d_theta_M, np.zeros_like(d_theta_M), np.zeros_like(d_theta_M))).T, 2)

In [ ]:
p_MConM = p_C - p_M
p_LConL = p_C - p_L
v_WConM = v_M + np.cross(omega_vec_M, p_MConM, axis=1)
v_WConMN = get_N_proj(v_WConM)

v_WConL = v_L + np.cross(omega_vec_L, p_LConL, axis=1)

p_LConLN = get_N_proj(p_C - p_L)

p_MConMN = get_N_proj(p_C - p_M)

In [ ]:
d_d_T_diff = diff_arr(d_T)
d_d_N_diff = diff_arr(d_N)

In [ ]:
v_S = np.matmul(
    np.transpose(T_hats, [0, 2, 1]),
    (v_WConM - v_WConL)
)
s_S = np.abs(v_S) # Slip speed

#### Friction coeficients

In [ ]:
stribeck_mu = stribeck(mu, mu, slip_speed/v_stiction)
stribeck_sign_L = np.sign(v_S).flatten()

In [ ]:
mu_SM = -stribeck_mu * stribeck_sign_L
mu_SL = stribeck_mu * stribeck_sign_L

#### Control inputs ($a_{Nd}$)

In [ ]:
dd_theta_Ld = scipy.interpolate.interp1d(d_times, debug['dd_theta_Lds'], axis=0)(l_times)
dd_d_Nd = scipy.interpolate.interp1d(d_times, debug['dd_d_Nds'], axis=0)(l_times)
dd_d_Td = scipy.interpolate.interp1d(d_times, debug['dd_d_Tds'], axis=0)(l_times)
dd_theta_Md = scipy.interpolate.interp1d(d_times, debug['dd_theta_Mds'], axis=0)(l_times)

### Outputs
These should all come directly from simulation, except for control forces.
#### Measured accelerations ($a_{LT}, a_{LN}, a_{MT}, a_{MN}, \ddot\theta$)

In [ ]:
a_L_raw = logger.data()[log_wrapper.entries_per_body*paper.link_idxs[-1] + 12:
                     log_wrapper.entries_per_body*paper.link_idxs[-1]+12+3,:]
a_L = np.expand_dims(a_L_raw.T, 2)
a_LT = get_T_proj(a_L)
a_LN = get_N_proj(a_L)

In [ ]:
a_M_raw = logger.data()[log_wrapper.entries_per_body*int(finger_body.index()) + 12:
                     log_wrapper.entries_per_body*int(finger_body.index())+12+3,:]
a_M = np.expand_dims(a_M_raw.T, 2)
a_MT = get_T_proj(a_M)
a_MN = get_N_proj(a_M)

In [ ]:
dd_theta_L = logger.data()[log_wrapper.entries_per_body*paper.link_idxs[-1] + 12 + 3,:]
dd_theta_M = logger.data()[log_wrapper.entries_per_body*int(finger_body.index()) + 12 + 3,:]

In [ ]:
a_s = diff_arr(slip_speed)

In [ ]:
a_M_diff = diff_vec(v_M)
a_MT_diff = get_T_proj(a_M_diff)
a_MN_diff = get_N_proj(a_M_diff)

#### Contact, object, and control forces on each object ($F_{NL}, F_{FL}, F_{ON}, F_{OT}, F_{NM}, F_{FM}, F_{CT}, F_{CN}$

##### Contact
Slightly different then just taking normal/tangent projections.

In [ ]:
F_contact_M_raw = logger.data()[log_wrapper.contact_entry_start_idx+0:log_wrapper.contact_entry_start_idx+3,:]
F_contact_M = np.expand_dims(F_contact_M_raw.T, 2)

In [ ]:
F_NM_vec = np.matmul(N_hat_proj_mat, F_contact_M)
F_FM_vec = F_contact_M - F_NM_vec

In [ ]:
F_NM_mag = np.linalg.norm(F_NM_vec, axis=1)
F_NM_sgn = np.sign(np.matmul(
    np.transpose(N_hats, [0, 2, 1]),
    F_NM_vec))
F_NM = F_NM_mag.flatten()*F_NM_sgn.flatten()

In [ ]:
F_FM_mag = np.linalg.norm(F_FM_vec, axis=1)
F_FM_sgn = np.sign(np.matmul(
    np.transpose(T_hats, [0, 2, 1]),
    F_FM_vec))
F_FM = F_FM_mag.flatten()*F_FM_sgn.flatten()

In [ ]:
F_FL = -F_FM
F_NL = -F_NM

##### Object

In [ ]:
joint_force_in_compliance_frame = logger.data()[
    log_wrapper.joint_entry_start_idx+6*(len(paper.joints) - 1):
    log_wrapper.joint_entry_start_idx+6*(len(paper.joints) - 1)+3,:]

In [ ]:
F_OT = joint_force_in_compliance_frame[1,:]
F_ON = joint_force_in_compliance_frame[2,:]

In [ ]:
joint_torque_in_compliance_frame = logger.data()[
    log_wrapper.joint_entry_start_idx+6*(len(paper.joints) - 1)+3:
    log_wrapper.joint_entry_start_idx+6*(len(paper.joints) - 1)+6,:]

In [ ]:
tau_O = joint_torque_in_compliance_frame[0,:]

##### Control

In [ ]:
F_CT_no_interp = np.array(debug['F_CTs'])
F_CT = scipy.interpolate.interp1d(d_times, F_CT_no_interp, axis=0)(l_times)
F_CN_no_interp = np.array(debug['F_CNs'])
F_CN = scipy.interpolate.interp1d(d_times, F_CN_no_interp, axis=0)(l_times)

In [ ]:
tau_M_no_interp = np.array(debug['taus'])
tau_M = scipy.interpolate.interp1d(d_times, tau_M_no_interp, axis=0)(l_times)

## Plotting settings

In [ ]:
lw = 5

In [ ]:
figsize = (16, 8)

In [ ]:
plot_supporting_plots = True

In [ ]:
# PROGRAMMING: Actually detect loss of contact, not just slipping
t_lose_contact = l_times[np.argmax(slip_speed > v_stiction)] #l_times[idx_lose_contact]

In [ ]:
def get_first_false_idx(arr):
    if len(arr.shape) > 1:
        nan_arr = np.any(np.isnan(arr), axis=0)
    else:
        nan_arr = np.isnan(arr)
    first_false_idx = None
    for i, val in enumerate(nan_arr):
        if not val:
            first_false_idx = i
            break
    return first_false_idx

In [ ]:
def idx_from_t(t):
    return np.argmax(l_times > t)

In [ ]:
first_false_idx = None
for i, val in enumerate(np.any(np.isnan(contact_point), axis=0)):
    if not val:
        first_false_idx = i
        break
idx_make_contact = i
t_make_contact = l_times[idx_make_contact]

In [ ]:
t_start = t_make_contact + 0.005
t_end = 0.116 # l_times[-1] #t_lose_contact #- 1e-4
idx_start = np.argmax(l_times > t_start)
idx_end = np.argmax(l_times > t_end)
idx_lose_contact = np.argmax(np.isnan(F_FL[idx_start:])) + idx_start

In [ ]:
def plot_eq(lhs, lhs_label, rhs, rhs_label, lhs_alpha=1, rhs_alpha=1, lw=5, finish_plotting=True, 
            plot_diff=True, figsize_=None, plot_err_as_pct=False, t_start=t_start, newline_in_title=False):
    if figsize_ is not None:
        plt.figure(figsize=figsize_)
    else:
        plt.figure(figsize=figsize)
    if isinstance(lhs, float) or isinstance(lhs, int):
        lhs = np.ones_like(l_times)*lhs
    if isinstance(rhs, float) or isinstance(rhs, int):
        rhs = np.ones_like(l_times)*rhs
    idx_start = np.argmax(l_times > t_start)
        
    b = np.broadcast(lhs, rhs)
    if np.count_nonzero(np.array(b.shape) > 1000) > 1:
        raise ValueError("Broadcast shape of lhs and rhs is {}, which is too large.".format(b.shape))
        
    plt.subplot(211)
    error = lhs[idx_start:idx_end]-rhs[idx_start:idx_end]
    if plot_err_as_pct:
        lhs_range = np.nanmax(lhs[idx_start:idx_end]) - np.nanmin(lhs[idx_start:idx_end])
        rhs_range = np.nanmax(rhs[idx_start:idx_end]) - np.nanmin(rhs[idx_start:idx_end])
        
        lhs_max_val = np.nanmax(np.abs(lhs[idx_start:idx_end]))
        rhs_max_val = np.nanmax(np.abs(rhs[idx_start:idx_end]))
        error /= np.nanmax([lhs_max_val, rhs_max_val])
        error *= 100
        
    plt.plot(l_times[idx_start:idx_end],
            error,
            linewidth=lw)
    plt.autoscale(enable=True, axis='x', tight=True)
    plt.xlabel("Time (s)")
    if plot_err_as_pct:
        plt.ylabel("Error (percent of max val)")
    else:
        plt.ylabel("Error")
    if newline_in_title:
        plt.title("Difference between {}\n and {}".format(lhs_label, rhs_label))
    else:
        plt.title("Difference between {} and {}".format(lhs_label, rhs_label))
    
    
    plt.subplot(212)
    plt.plot(l_times[idx_start:idx_end],
             lhs[idx_start:idx_end],
             label=lhs_label, linewidth=lw, color='lightskyblue', zorder=0, alpha=lhs_alpha)
    plt.plot(l_times[idx_start:idx_end],
             rhs[idx_start:idx_end],
             label=rhs_label, linewidth=lw, color='b', linestyle='--', zorder=1, alpha=rhs_alpha)
    plt.autoscale(enable=True, axis='x', tight=True)
    
    
#     plt.axvline(t_make_contact, color='green', linewidth=lw)
#     plt.axvline(t_lose_contact, color='red', linewidth=lw)
    
    plt.legend()
    plt.xlabel("Time (s)")
    
    if finish_plotting:
        plt.show()

In [ ]:
errors = []
max_errors = []
med_errors = []
labels = []

In [ ]:
def update_error(lhs, lhs_label, rhs, rhs_label, newline_in_label=False):
    error = ((lhs) - (rhs))[idx_start:idx_end]
    
    label = r"$\left(" + lhs_label.replace("$", "") + r"\right) "
    if newline_in_label:
        label += "$\n$"
    label += "- \left(" + rhs_label.replace("$", "") + r"\right)$"
    label_idx = None
    for i, l in enumerate(labels):
        if l == label:
            label_idx = i
            break
    max_error = np.nanmax(np.abs(error))
    med_error = np.median(np.abs(error[np.logical_not(np.isnan(error))]))
    if label_idx is None:
        errors.append(error)
        labels.append(label)
        max_errors.append(max_error)
        med_errors.append(med_error)
    else:
        errors[label_idx] = error
        max_errors[label_idx] = max_error
        med_errors[label_idx] = med_error

## Plots (environmental constraints)
There are things where I expect them to be satisfied no matter what, no matter the succes my controller.

### $m_La_{LT} = F_{FL} + F_{GT} +F_{OT}$
*If this plot is correct:*
The free body diagram is correct in thet $\hat T$ direction on the link.

In [ ]:
lhs_label = r'$m_La_{LT}$'
rhs_label = r'$F_{FL} + F_{GT} +F_{OT}$'

lhs = m_L*a_LT

rhs = F_FL+F_GT+F_OT

plot_eq(lhs, lhs_label, rhs, rhs_label)
update_error(lhs, lhs_label, rhs, rhs_label)

### $m_La_{LN} = F_{NL} + F_{GN} +F_{ON}$
*If this plot is correct:*
The free body diagram is correct in thet $\hat N$ direction on the link.

In [ ]:
lhs_label = r'$m_La_{LN}$'
rhs_label = r'$F_{NL} + F_{GN} +F_{ON}$'

lhs = m_L*a_LN

rhs = F_NL + F_GN + F_ON

plot_eq(lhs, lhs_label, rhs, rhs_label)
update_error(lhs, lhs_label, rhs, rhs_label)

### $m_Ma_{MT} = F_{FM} + F_{CT}$
*If this plot is correct:*
The free body diagram is correct in thet $\hat T$ direction on the manipulator.

In [ ]:
lhs_label = r'$m_Ma_{MT}$'
rhs_label = r'$F_{FM} + F_{CT}$'

lhs = m_M*a_MT

rhs = F_FM + F_CT

plot_eq(lhs, lhs_label, rhs, rhs_label)
update_error(lhs, lhs_label, rhs, rhs_label)

### $m_Ma_{MN} = F_{NM} + F_{CN}$
*If this plot is correct:*
The free body diagram is correct in thet $\hat TN direction on the manipulator.

In [ ]:
lhs_label = r'$m_Ma_{MN}$'
rhs_label = r'$ F_{NM} + F_{CN}$'

lhs = m_M*a_MN

rhs = F_NM+F_CN

plot_eq(lhs, lhs_label, rhs, rhs_label)
update_error(lhs, lhs_label, rhs, rhs_label)

### $I_L\ddot\theta_L = -\frac{w_L}{2}F_{ON} - (p_{CN}-p_{LN})F_{FL} + (p_{CT}-p_{LT})F_{NL}+\tau_O$
*If this plot is correct:* The moment balance on the link is correct.

In [ ]:
lhs_label = r'$I_L\ddot\theta$'
rhs_label = r'$-\frac{w_L}{2}F_{ON} - (p_{CN}-p_{LN})F_{FL} + (p_{CT}-p_{LT})F_{NL}+\tau_O$'

lhs = I_L*dd_theta_L

rhs = (-w_L/2)*F_ON - (p_CN-p_LN)*F_FL + (p_CT-p_LT)*F_NL + tau_O

plot_eq(lhs, lhs_label, rhs, rhs_label)
update_error(lhs, lhs_label, rhs, rhs_label)

### $I_M\ddot\theta_M = \tau_M-F_{FM}(p_{CN}-p_{MN})$
*If this plot is correct:* The moment balance on the manipulator is correct.

In [ ]:
lhs = I_M*dd_theta_M
lhs_label = r'$I_M\ddot\theta_M$'
rhs = tau_M-F_FM*(p_CN-p_MN)
rhs_label = r"$\tau_M-F_{FM}(p_{CN}-p_{MN})$"
plot_eq(lhs, lhs_label, rhs, rhs_label)
update_error(lhs, lhs_label, rhs, rhs_label)

### $F_{NL} = -F_{NM}$
*If this plot is correct:* Newton's 3rd law is correction applied to the normal forces.

In [ ]:
lhs_label = r'$F_{NL}$'
rhs_label = r'$-F_{NM}$'

lhs = F_NL

rhs = -F_NM

plot_eq(lhs, lhs_label, rhs, rhs_label)
update_error(lhs, lhs_label, rhs, rhs_label)

### $-\ddot\theta_L h_L/2 - \ddot\theta_L r + \dot\theta_L^2 w_L/2 - a_{LT} + a_{MT} = -\ddot\theta_L {d}_N + \ddot{d}_T - \dot\theta_L^2 {d}_T - 2\dot\theta_L \dot{d}_N$
*If this plot is correct:* The derivatives for  $\vec d$  are correct.

In [ ]:
lhs = -dd_theta_L*(h_L/2+r) + d_theta_L**2*w_L/2 - a_LT + a_MT_diff
lhs_label = r"$-\ddot\theta_L h_L/2 - \ddot\theta_L r + \dot\theta_L^2 w_L/2 - a_{LT} + a_{MT}$"

rhs = -dd_theta_L*d_N + dd_d_T - d_theta_L**2*d_T - 2*d_theta_L*d_d_N
rhs_label = r"$-\ddot\theta_L {d}_N + \ddot{d}_T - \dot\theta_L^2 {d}_T - 2\dot\theta_L \dot{d}_N$"

plot_eq(lhs, lhs_label, rhs, rhs_label)
update_error(lhs, lhs_label, rhs, rhs_label, newline_in_label=True)

### $-\ddot\theta_L w_L/2 - \dot\theta_L^2 h_L/2 - \dot\theta_L^2 r - a_{LN} + a_{MN} = \ddot\theta_L {d}_T + \ddot{d}_N - \dot\theta_L^2 {d}_N + 2 \dot\theta_L \dot{d}_T$
*If this plot is correct:* The derivatives for  $\vec d$  are correct.

In [ ]:
lhs = -dd_theta_L*w_L/2 - d_theta_L**2*h_L/2 - d_theta_L**2*r - a_LN + a_MN
lhs_label = r"$-\ddot\theta_L w_L/2 - \dot\theta_L^2 h_L/2 - \dot\theta_L^2 r - a_{LN} + a_{MN}$ (differentiating geometry)"

rhs = dd_theta_L*d_T + dd_d_N - d_theta_L**2*d_N + 2*d_theta_L*d_d_T
rhs_label = r"$\ddot\theta_L {d}_T + \ddot{d}_N - \dot\theta_L^2 {d}_N + 2 \dot\theta_L \dot{d}_T$ (differentiating vector)"

plot_eq(lhs, lhs_label, rhs, rhs_label, newline_in_title=True)

lhs_label = r"$-\ddot\theta_L w_L/2 - \dot\theta_L^2 h_L/2 - \dot\theta_L^2 r - a_{LN} + a_{MN}$"
rhs_label = r"$\ddot\theta_L {d}_T + \ddot{d}_N - \dot\theta_L^2 {d}_N + 2 \dot\theta_L \dot{d}_T$"
update_error(lhs, lhs_label, rhs, rhs_label, newline_in_label=True)
plt.show()

### $F_{FL}=\mu_{SL}F_{NL}$
*If this plot is correct:* The friction model used matches what Drake uses.

In [ ]:
lhs = F_FL
lhs_label = r"$F_{FL}$"

rhs = mu_SL*F_NL
rhs_label = r"$\mu_{SL}F_{NL}$"

plot_eq(lhs, lhs_label, rhs, rhs_label)

update_error(lhs, lhs_label, rhs, rhs_label)
plt.show()

### $F_{FM}=\mu_{SM}F_{NL}$
*If this plot is correct:* The friction model used matches what Drake uses.

In [ ]:
lhs = F_FM
lhs_label = r"$F_{FM}$"

rhs = mu_SM*F_NL
rhs_label = r"$\mu_{SM}F_{NL}$"

plot_eq(lhs, lhs_label, rhs, rhs_label)

update_error(lhs, lhs_label, rhs, rhs_label)
plt.show()

## Plot errors

In [ ]:
plt.figure(figsize=(16,16))
plt.barh(labels[::-1], max_errors[::-1], label="Maximum error")
plt.barh(labels[::-1], med_errors[::-1], label="Median error")
plt.axvline(1e-5, color='k', linestyle='--', label="1e-5")
plt.xscale("log")
plt.xlabel("Log(error)")
plt.ylabel("Equation")
plt.legend()
plt.xlim(10**-16, 10**3)
plt.show()

## Plots (artificial constraints)
These will only be satisfied if the controller is successful.

### $\ddot{d}_N = \ddot{d}_{Nd}$

In [ ]:
lhs_label = r'$\ddot{d}_N$'
rhs_label = r'$\ddot{d}_{Nd}$'

lhs = dd_d_N

rhs = dd_d_Nd

plot_eq(lhs, lhs_label, rhs, rhs_label)

### $\ddot{d}_T = \ddot{d}_{Td}$

In [ ]:
lhs_label = r'$\ddot{d}_T$'
rhs_label = r'$\ddot{d}_{Td}$'

# lhs = moving_average(dd_d_T, 100)
lhs = -dd_theta_L*(h_L/2+r) + d_theta_L**2*w_L/2 - a_LT + a_MT_diff - (dd_theta_L*d_N - d_theta_L**2*d_T - 2*d_theta_L*d_d_N)

rhs = dd_d_Td

plot_eq(lhs, lhs_label, rhs, rhs_label)

### $\ddot\theta_L=\ddot\theta_{Ld}$

In [ ]:
lhs_label = r'$\ddot\theta_L$'
rhs_label = r'$\ddot\theta_{Ld}$'

lhs = dd_theta_L

rhs = dd_theta_Ld

plot_eq(lhs, lhs_label, rhs, rhs_label)

### $\ddot\theta_M=\ddot\theta_{Md}$

In [ ]:
lhs_label = r'$\ddot\theta_M$'
rhs_label = r'$\ddot\theta_{Md}$'

lhs = dd_theta_M

rhs = dd_theta_Md

plot_eq(lhs, lhs_label, rhs, rhs_label)

# Control targets
These are checking that where feedback control is used, we actually hit the target.
## $d_T$

In [ ]:
plt.figure(figsize=(16,16))
plt.subplot(311)
plt.plot(l_times[idx_start:idx_end], d_T[idx_start:idx_end], color='r', linestyle='-', linewidth=5)
plt.axhline(finger_ctrlr.d_Td, color='k', linestyle='--', linewidth=5)
plt.autoscale(enable=True, axis='x', tight=True)

plt.subplot(312)
plt.plot(l_times[idx_start:idx_end], finger_ctrlr.d_Td-d_T[idx_start:idx_end],
         color='g', linestyle='-', linewidth=5)
plt.autoscale(enable=True, axis='x', tight=True)

plt.subplot(313)
plt.plot(l_times[idx_start:idx_end], dd_d_Td[idx_start:idx_end]*0.05, linestyle='-', linewidth=5)
plt.autoscale(enable=True, axis='x', tight=True)
plt.show()

In [ ]:
lw=5
plt.figure(figsize=(16, 8))
plt.axhline(finger_ctrlr.d_Td, linestyle='--', color='gray', linewidth=lw, label="Desired $d_T$")
plt.plot(l_times, d_T, label='$d_T$', linewidth=lw)
plt.axhline(-(0), label='Bounds on $d_T$ (0 and link width)', linestyle='--', color='k', linewidth=lw)
plt.axhline(-(paper.link_width), linestyle='--', color='k', linewidth=lw)
plt.legend()
plt.xlabel("Time (seconds)")
plt.ylabel("Distance (m)")
plt.autoscale(enable=True, axis='x', tight=True)
plt.xlim(t_start, t_end)
plt.show()

## $\dot\theta_L$

In [ ]:
plt.figure(figsize=(16,16))
plt.subplot(311)
plt.plot(l_times[idx_start:idx_end], d_theta_L[idx_start:idx_end], color='r', linestyle='-', linewidth=5)
plt.axhline(finger_ctrlr.d_theta_Ld, color='k', linestyle='--', linewidth=5)
plt.autoscale(enable=True, axis='x', tight=True)

plt.subplot(312)
plt.plot(l_times[idx_start:idx_end], finger_ctrlr.d_theta_Ld-d_theta_L[idx_start:idx_end],
         color='g', linestyle='-', linewidth=5)
plt.autoscale(enable=True, axis='x', tight=True)

plt.subplot(313)
plt.plot(l_times[idx_start:idx_end], dd_theta_Ld[idx_start:idx_end]*0.05, linestyle='-', linewidth=5)
plt.autoscale(enable=True, axis='x', tight=True)
plt.show()

# Control estimates

In [ ]:
F_OT_debug = scipy.interpolate.interp1d(d_times, debug['F_OTs'], axis=0)(l_times)
F_ON_debug = scipy.interpolate.interp1d(d_times, debug['F_ONs'], axis=0)(l_times)
tau_O_debug = scipy.interpolate.interp1d(d_times, debug['tau_Os'], axis=0)(l_times)

In [ ]:
plot_eq(F_OT_debug, "debug", F_OT, "actual")

In [ ]:
plot_eq(F_ON_debug, "debug", F_ON, "actual")

In [ ]:
plot_eq(tau_O_debug, "debug", tau_O, "actual")

# Other useful plots

## Friction plots

In [ ]:
lw=5
plt.figure(figsize=(16,8))
plt.plot(l_times[idx_start:idx_end],
         abs(F_FL[idx_start:idx_end]), label=r"$\left|F_{FL}\right|$", linewidth=lw, color='lightskyblue', zorder=0)
plt.plot(l_times[idx_start:idx_end],
         abs(F_FM[idx_start:idx_end]), label=r"$\left|F_{FM}\right|$", linewidth=lw, color='b', linestyle='--', zorder=1)
plt.plot(l_times[idx_start:idx_end],
         abs(F_NL[idx_start:idx_end])*constants.FRICTION, label=r"$\mu\left|F_{NL}\right|$", linewidth=lw, color='k', linestyle=':')

plt.autoscale(enable=True, axis='x', tight=True)
plt.legend()
plt.xlabel("Time")
plt.show()

In [ ]:
lw=5
plt.figure(figsize=(16, 8))
plt.plot(l_times[idx_start:idx_end], slip_speed[idx_start:idx_end], label="slip speed", linewidth=5)
plt.axhline(v_stiction, linestyle="--", color="k", label="$v_{stiction}$", linewidth=5)
plt.xlabel("Time (seconds)")
plt.ylabel("Speed (m/s)")
# plt.ylim(0, 2*v_stiction)
plt.autoscale(enable=True, axis='x', tight=True)
plt.legend()
plt.show()

## Joint torques

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(l_times[idx_start:idx_end], 
         joint_torque_in_compliance_frame[0,idx_start:idx_end], linewidth=5, color='r', label=r"\tau_x")
plt.plot(l_times[idx_start:idx_end], 
         joint_torque_in_compliance_frame[1,idx_start:idx_end], linewidth=5, color='g', label=r"\tau_y")
plt.plot(l_times[idx_start:idx_end], 
         joint_torque_in_compliance_frame[2,idx_start:idx_end], linewidth=5, color='b', label=r"\tau_z")
plt.autoscale(enable=True, axis='x', tight=True)